In [1]:
import sys

sys.path.append('../input/shopee-competition-utils')
sys.path.insert(0,'../input/pytorch-image-models')

In [2]:
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.nn import Parameter
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations
from albumentations.pytorch.transforms import ToTensorV2

from custom_scheduler import ShopeeScheduler
from custom_activation import replace_activations, Mish
from custom_optimizer import Ranger

import math
import cv2
import timm
import os
import random
import gc

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold
from sklearn.neighbors import NearestNeighbors
from tqdm.notebook import tqdm

In [3]:
class CFG: 
    
    DATA_DIR = '../input/shopee-product-matching/train_images'
    TRAIN_CSV = '../input/shopee-product-matching/train.csv'

    # data augmentation
    IMG_SIZE = 512
    MEAN = [0.485, 0.456, 0.406]
    STD = [0.229, 0.224, 0.225]

    SEED = 2021

    # data split
    N_SPLITS = 5
    TEST_FOLD = 0
    VALID_FOLD = 1

    EPOCHS = 8
    BATCH_SIZE = 8

    NUM_WORKERS = 4
    DEVICE = 'cuda:1'

    CLASSES = 6609 
    SCALE = 30
    MARGINS = [0.5,0.6,0.7,0.8,0.9]
    MARGIN = 0.5

    BEST_THRESHOLD = 0.19
    BEST_THRESHOLD_MIN2 = 0.225

    MODEL_NAME = 'resnet50'
    MODEL_NAMES = ['resnet50','resnext50_32x4d','densenet121','efficientnet_b3','eca_nfnet_l0']
    LOSS_MODULE = 'arc'
    LOSS_MODULES = ['arc','curricular']
    USE_ARCFACE = True
    MODEL_PATH_PREFIX = '../input/image-model-trained/'
    EMB_PATH_PREFIX = '../input/image-embeddings/'
    USE_EMBEDDING = True
    MODEL_PATH = f'{MODEL_NAME}_{LOSS_MODULE}_face_epoch_8_bs_8_margin_{MARGIN}.pt'
    FC_DIM = 512
    SCHEDULER_PARAMS = {
            "lr_start": 1e-5,
            "lr_max": 1e-5 * 32,
            "lr_min": 1e-6,
            "lr_ramp_ep": 5,
            "lr_sus_ep": 0,
            "lr_decay": 0.8,
        }

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True # set True to be faster

seed_everything(CFG.SEED)

In [5]:
def get_test_transforms():
    return albumentations.Compose(
        [
            albumentations.Resize(CFG.IMG_SIZE,CFG.IMG_SIZE,always_apply=True),
            albumentations.Normalize(mean=CFG.MEAN, std=CFG.STD),
            ToTensorV2(p=1.0)
        ]
    )

In [6]:
class ShopeeImageDataset(torch.utils.data.Dataset):
    """for validating and test
    """
    def __init__(self,df, transform = None):
        self.df = df 
        self.root_dir = CFG.DATA_DIR
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self,idx):

        row = self.df.iloc[idx]

        img_path = os.path.join(self.root_dir,row.image)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = row.label_group

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']     
                
        return image,torch.tensor(1)

## ArcMarginProduct

In [7]:
class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False, ls_eps=0.0):
        print('Using Arc Face')
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.ls_eps = ls_eps  # label smoothing
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device=CFG.DEVICE)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output, nn.CrossEntropyLoss()(output,label)

## CurricularFace

In [8]:
'''
credit : https://github.com/HuangYG123/CurricularFace/blob/8b2f47318117995aa05490c05b455b113489917e/head/metrics.py#L70
'''

def l2_norm(input, axis = 1):
    norm = torch.norm(input, 2, axis, True)
    output = torch.div(input, norm)

    return output

class CurricularFace(nn.Module):
    def __init__(self, in_features, out_features, s = 30, m = 0.50):
        super(CurricularFace, self).__init__()

        print('Using Curricular Face')

        self.in_features = in_features
        self.out_features = out_features
        self.m = m
        self.s = s
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.threshold = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m
        self.kernel = nn.Parameter(torch.Tensor(in_features, out_features))
        self.register_buffer('t', torch.zeros(1))
        nn.init.normal_(self.kernel, std=0.01)

    def forward(self, embbedings, label):
        embbedings = l2_norm(embbedings, axis = 1)
        kernel_norm = l2_norm(self.kernel, axis = 0)
        cos_theta = torch.mm(embbedings, kernel_norm)
        cos_theta = cos_theta.clamp(-1, 1)  # for numerical stability
        with torch.no_grad():
            origin_cos = cos_theta.clone()
        target_logit = cos_theta[torch.arange(0, embbedings.size(0)), label].view(-1, 1)

        sin_theta = torch.sqrt(1.0 - torch.pow(target_logit, 2))
        cos_theta_m = target_logit * self.cos_m - sin_theta * self.sin_m #cos(target+margin)
        mask = cos_theta > cos_theta_m
        final_target_logit = torch.where(target_logit > self.threshold, cos_theta_m, target_logit - self.mm)

        hard_example = cos_theta[mask]
        with torch.no_grad():
            self.t = target_logit.mean() * 0.01 + (1 - 0.01) * self.t
        cos_theta[mask] = hard_example * (self.t + hard_example)
        cos_theta.scatter_(1, label.view(-1, 1).long(), final_target_logit)
        output = cos_theta * self.s
        return output, nn.CrossEntropyLoss()(output,label)

In [9]:
class ShopeeModel(nn.Module):

    def __init__(
        self,
        n_classes = CFG.CLASSES,
        model_name = CFG.MODEL_NAME,
        fc_dim = CFG.FC_DIM,
        margin = CFG.MARGIN,
        scale = CFG.SCALE,
        use_fc = True,
        pretrained = True,
        use_arcface = CFG.USE_ARCFACE):

        super(ShopeeModel,self).__init__()
        print(f'Building Model Backbone for {model_name} model, margin = {margin}')

        self.backbone = timm.create_model(model_name, pretrained=pretrained)

        if 'efficientnet' in model_name:
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()
        
        elif 'resnet' in model_name:
            final_in_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif 'resnext' in model_name:
            final_in_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif 'densenet' in model_name:
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif 'nfnet' in model_name:
            final_in_features = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity()
            self.backbone.head.global_pool = nn.Identity()

        self.pooling =  nn.AdaptiveAvgPool2d(1)

        self.use_fc = use_fc

        if use_fc:
            self.dropout = nn.Dropout(p=0.0)
            self.fc = nn.Linear(final_in_features, fc_dim)
            self.bn = nn.BatchNorm1d(fc_dim)
            self._init_params()
            final_in_features = fc_dim

        if use_arcface:
            self.final = ArcMarginProduct(final_in_features, 
                                                n_classes, 
                                                s=scale, 
                                                m=margin)
        else:
            self.final = CurricularFace(final_in_features, 
                                                n_classes, 
                                                s=scale, 
                                                m=margin)

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, image, label):
        feature = self.extract_feat(image)
        logits = self.final(feature,label)
        return logits

    def extract_feat(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.pooling(x).view(batch_size, -1)

        if self.use_fc:
            x = self.dropout(x)
            x = self.fc(x)
            x = self.bn(x)
        return x


In [10]:
def read_dataset():
    df = pd.read_csv(CFG.TRAIN_CSV)
    df['matches'] = df.label_group.map(df.groupby('label_group').posting_id.agg('unique').to_dict())
    df['matches'] = df['matches'].apply(lambda x: ' '.join(x))

    gkf = GroupKFold(n_splits=CFG.N_SPLITS)
    df['fold'] = -1
    for i, (train_idx, valid_idx) in enumerate(gkf.split(X=df, groups=df['label_group'])):
        df.loc[valid_idx, 'fold'] = i

    labelencoder= LabelEncoder()
    df['label_group'] = labelencoder.fit_transform(df['label_group'])

    train_df = df[df['fold']!=CFG.TEST_FOLD].reset_index(drop=True)
    train_df = train_df[train_df['fold']!=CFG.VALID_FOLD].reset_index(drop=True)
    valid_df = df[df['fold']==CFG.VALID_FOLD].reset_index(drop=True)
    test_df = df[df['fold']==CFG.TEST_FOLD].reset_index(drop=True)

    train_df['label_group'] = labelencoder.fit_transform(train_df['label_group'])

    return train_df, valid_df, test_df

In [11]:
def precision_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    precision = intersection / len_y_pred
    return precision

def recall_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_true = y_true.apply(lambda x: len(x)).values
    recall = intersection / len_y_true
    return recall

def f1_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    len_y_true = y_true.apply(lambda x: len(x)).values
    f1 = 2 * intersection / (len_y_pred + len_y_true)
    return f1

In [12]:
def get_image_embeddings(df, model):

    image_dataset = ShopeeImageDataset(df,transform=get_test_transforms())
    image_loader = torch.utils.data.DataLoader(
        image_dataset,
        batch_size=CFG.BATCH_SIZE,
        pin_memory=True,
        num_workers = CFG.NUM_WORKERS,
        drop_last=False
    )

    embeds = []
    with torch.no_grad():
        for img,label in tqdm(image_loader): 
            img = img.to(CFG.DEVICE)
            label = label.to(CFG.DEVICE)
            feat,_ = model(img,label)
            image_embeddings = feat.detach().cpu().numpy()
            embeds.append(image_embeddings)
    
    del model
    image_embeddings = np.concatenate(embeds)
    print(f'Our image embeddings shape is {image_embeddings.shape}')
    del embeds
    gc.collect()
    return image_embeddings

In [13]:
def get_image_neighbors(df, embeddings, threshold = 0.2, min2 = False):

    nbrs = NearestNeighbors(n_neighbors = 50, metric = 'cosine')
    nbrs.fit(embeddings)
    distances, indices = nbrs.kneighbors(embeddings)

    predictions = []
    for k in range(embeddings.shape[0]):
        if min2:
            idx = np.where(distances[k,] < CFG.BEST_THRESHOLD)[0]
            ids = indices[k,idx]
            if len(ids) <= 1 and distances[k,1] < threshold:
                ids = np.append(ids,indices[k,1])
        else:
            idx = np.where(distances[k,] < threshold)[0]
            ids = indices[k,idx]
        posting_ids = ' '.join(df['posting_id'].iloc[ids].values)
        predictions.append(posting_ids)
        
    df['pred_matches'] = predictions
    df['f1'] = f1_score(df['matches'], df['pred_matches'])
    df['recall'] = recall_score(df['matches'], df['pred_matches'])
    df['precision'] = precision_score(df['matches'], df['pred_matches'])
    
    del nbrs, distances, indices
    gc.collect()
    return df

In [14]:
def search_best_threshold(valid_df,model=0):
    search_space = np.arange(10, 50, 1)
    if CFG.USE_EMBEDDING:
        VALID_EMBEDDING_PATH = CFG.EMB_PATH_PREFIX + CFG.MODEL_PATH[:-3] + '_valid_embed.csv'
        valid_embeddings = np.loadtxt(VALID_EMBEDDING_PATH, delimiter=',')
    else:
        valid_embeddings = get_image_embeddings(valid_df, model)

    print("Searching best threshold...")
    best_f1_valid = 0.
    best_threshold = 0.
    for i in search_space:
        threshold = i / 100
        valid_df = get_image_neighbors(valid_df, valid_embeddings, threshold=threshold)
        valid_f1 = valid_df.f1.mean()
        valid_recall = valid_df.recall.mean()
        valid_precision = valid_df.precision.mean()
        print(f"threshold = {threshold} -> f1 score = {valid_f1}, recall = {valid_recall}, precision = {valid_precision}")
        if (valid_f1 > best_f1_valid):
            best_f1_valid = valid_f1
            best_threshold = threshold

    print("Best threshold =", best_threshold)
    print("Best f1 score =", best_f1_valid)
    CFG.BEST_THRESHOLD = best_threshold

    # phase 2 search
    print("________________________________")
    print("Searching best min2 threshold...")
    search_space = np.arange(CFG.BEST_THRESHOLD * 100, CFG.BEST_THRESHOLD * 100 + 20, 0.5)

    best_f1_valid = 0.
    best_threshold = 0.

    for i in search_space:
        threshold = i / 100
        valid_df = get_image_neighbors(valid_df, valid_embeddings, threshold=threshold,min2=True)

        valid_f1 = valid_df.f1.mean()
        valid_recall = valid_df.recall.mean()
        valid_precision = valid_df.precision.mean()

        print(f"min2 threshold = {threshold} -> f1 score = {valid_f1}, recall = {valid_recall}, precision = {valid_precision}")

        if (valid_f1 > best_f1_valid):
            best_f1_valid = valid_f1
            best_threshold = threshold

    print("Best min2 threshold =", best_threshold)
    print("Best f1 score after min2 =", best_f1_valid)
    CFG.BEST_THRESHOLD_MIN2 = best_threshold


In [15]:
def search_inb_threshold(valid_df,valid_embeddings):
    search_space = np.arange(10, 50, 1)
    print("Searching best threshold...")
    best_f1_valid = 0.
    best_threshold = 0.
    for i in search_space:
        threshold = i / 100
        valid_df = get_image_neighbors(valid_df, valid_embeddings, threshold=threshold)
        valid_f1 = valid_df.f1.mean()
        valid_recall = valid_df.recall.mean()
        valid_precision = valid_df.precision.mean()
        print(f"threshold = {threshold} -> f1 score = {valid_f1}, recall = {valid_recall}, precision = {valid_precision}")
        if (valid_f1 > best_f1_valid):
            best_f1_valid = valid_f1
            best_threshold = threshold

    print("Best threshold =", best_threshold)
    print("Best f1 score =", best_f1_valid)
    CFG.BEST_THRESHOLD = best_threshold

    # phase 2 search
    print("________________________________")
    print("Searching best min2 threshold...")
    search_space = np.arange(CFG.BEST_THRESHOLD * 100, CFG.BEST_THRESHOLD * 100 + 20, 0.5)

    best_f1_valid = 0.
    best_threshold = 0.

    for i in search_space:
        threshold = i / 100
        valid_df = get_image_neighbors(valid_df, valid_embeddings, threshold=threshold,min2=True)

        valid_f1 = valid_df.f1.mean()
        valid_recall = valid_df.recall.mean()
        valid_precision = valid_df.precision.mean()

        print(f"min2 threshold = {threshold} -> f1 score = {valid_f1}, recall = {valid_recall}, precision = {valid_precision}")

        if (valid_f1 > best_f1_valid):
            best_f1_valid = valid_f1
            best_threshold = threshold

    print("Best min2 threshold =", best_threshold)
    print("Best f1 score after min2 =", best_f1_valid)
    CFG.BEST_THRESHOLD_MIN2 = best_threshold


In [16]:
def run_test_all():
    train_df, valid_df, test_df = read_dataset()
    for i in range(len(CFG.LOSS_MODULES)):
        CFG.LOSS_MODULE = CFG.LOSS_MODULES[i]
        if 'arc' in CFG.LOSS_MODULE:
            CFG.USE_ARCFACE = True
        else:
            CFG.USE_ARCFACE = False
        for j in range(len(CFG.MODEL_NAMES)):
            CFG.MODEL_NAME = CFG.MODEL_NAMES[j]
            for k in range(len(CFG.MARGINS)):
                CFG.MARGIN = CFG.MARGINS[k]
                model = ShopeeModel(model_name = CFG.MODEL_NAME,
                                    margin = CFG.MARGIN,
                                    use_arcface = CFG.USE_ARCFACE)
                model.eval()
                model = replace_activations(model, torch.nn.SiLU, Mish())
                CFG.MODEL_PATH = f'{CFG.MODEL_NAME}_{CFG.LOSS_MODULE}_face_epoch_8_bs_8_margin_{CFG.MARGIN}.pt'
                model.load_state_dict(torch.load(CFG.MODEL_PATH))
                model = model.to(CFG.DEVICE)
                search_best_threshold(valid_df,model)
                test_embeddings = get_image_embeddings(test_df, model)
                test_df = get_image_neighbors(test_df, test_embeddings, threshold=CFG.BEST_THRESHOLD_MIN2, min2 = True)


In [17]:
def run_test():
    train_df, valid_df, test_df = read_dataset()
    if 'arc' in CFG.LOSS_MODULE:
        CFG.USE_ARCFACE = True
    else:
        CFG.USE_ARCFACE = False
    if CFG.USE_EMBEDDING:
        model = 0
        search_best_threshold(valid_df,model)
        CFG.MODEL_PATH = f'{CFG.MODEL_NAME}_{CFG.LOSS_MODULE}_face_epoch_8_bs_8_margin_{CFG.MARGIN}.pt'
        TEST_EMBEDDING_PATH = CFG.EMB_PATH_PREFIX + CFG.MODEL_PATH[:-3] + '_test_embed.csv'
        test_embeddings = np.loadtxt(TEST_EMBEDDING_PATH, delimiter=',')
    else:
        model = ShopeeModel(model_name = CFG.MODEL_NAME,
                            margin = CFG.MARGIN,
                            use_arcface = CFG.USE_ARCFACE)
        model.eval()
        model = replace_activations(model, torch.nn.SiLU, Mish())
        CFG.MODEL_PATH = f'{CFG.MODEL_NAME}_{CFG.LOSS_MODULE}_face_epoch_8_bs_8_margin_{CFG.MARGIN}.pt'
        print(CFG.MODEL_PATH)
        model.load_state_dict(torch.load(CFG.MODEL_PATH))
        model = model.to(CFG.DEVICE)

        search_best_threshold(valid_df,model)
        test_embeddings = get_image_embeddings(test_df, model)

    test_df = get_image_neighbors(test_df, test_embeddings, threshold=CFG.BEST_THRESHOLD)
    test_f1 = test_df.f1.mean()
    test_recall = test_df.recall.mean()
    test_precision = test_df.precision.mean()
    print(f'Test f1 score = {test_f1}, recall = {test_recall}, precision = {test_precision}')

    test_df = get_image_neighbors(test_df, test_embeddings, threshold=CFG.BEST_THRESHOLD_MIN2, min2 = True)
    test_f1 = test_df.f1.mean()
    test_recall = test_df.recall.mean()
    test_precision = test_df.precision.mean()
    print(f'Test f1 score after min2 = {test_f1}, recall = {test_recall}, precision = {test_precision}')

## Run test

Parameters:
+ `CFG.MARGIN = [0.5,0.6,0.7,0.8,0.9]`
+ `CFG.MODEL_NAME = ['resnet50','resnext50_32x4d','densenet121','efficientnet_b3','eca_nfnet_l0']`
+ `CFG.LOSS_MODULE = ['arc','curricular']`

In [18]:
# CFG.LOSS_MODULE = CFG.LOSS_MODULES[0]
# CFG.MODEL_NAME = CFG.MODEL_NAMES[0]
# CFG.MARGIN = CFG.MARGINS[2]
# run_test()

In [19]:
def blend_neighborhood(emb, match_index_lst, similarities_lst):
    new_emb = emb.copy()
    for i in range(emb.shape[0]):
        cur_emb = emb[match_index_lst[i]]
        weights = np.expand_dims(similarities_lst[i], 1)
        new_emb[i] = (cur_emb * weights).sum(axis=0)
    new_emb = normalize(new_emb, axis=1)
    return new_emb

def iterative_neighborhood_blending(emb, threshes):
    for thresh in threshes:
        match_index_lst, similarities_lst = neighborhood_search(emb, thresh)
        emb = blend_neighborhood(emb, match_index_lst, similarities_lst)
    return match_index_lst

In [20]:
def blend_neighborhood(df, embeddings):
    nbrs = NearestNeighbors(n_neighbors = 50, metric = 'cosine')
    nbrs.fit(embeddings)
    distances, indices = nbrs.kneighbors(embeddings)
    similarities = 1 - distances

    predictions = []
    new_emb = embeddings.copy()
    for k in range(embeddings.shape[0]):
        idx = np.where(distances[k,] < CFG.BEST_THRESHOLD)[0]
        ids = indices[k,idx]
        if len(ids) <= 1 and distances[k,1] < CFG.BEST_THRESHOLD_MIN2:
            ids = np.append(ids,indices[k,1])
        cur_emb = embeddings[ids]
        weights = np.expand_dims(similarities[k,0:len(ids)], 1)
        new_emb[k] = (cur_emb * weights).sum(axis=0)
        posting_ids = ' '.join(df['posting_id'].iloc[ids].values)
        predictions.append(posting_ids)
        
    df['pred_matches'] = predictions
    df['f1'] = f1_score(df['matches'], df['pred_matches'])
    df['recall'] = recall_score(df['matches'], df['pred_matches'])
    df['precision'] = precision_score(df['matches'], df['pred_matches'])

    f1 = df.f1.mean()
    recall = df.recall.mean()
    precision = df.precision.mean()
    print(f'Test f1 score = {f1}, recall = {recall}, precision = {precision}')

    return new_emb

In [21]:
CFG.LOSS_MODULE = CFG.LOSS_MODULES[0]
CFG.MODEL_NAME = CFG.MODEL_NAMES[0]
CFG.MARGIN = CFG.MARGINS[0]
CFG.MODEL_PATH = f'{CFG.MODEL_NAME}_{CFG.LOSS_MODULE}_face_epoch_8_bs_8_margin_{CFG.MARGIN}.pt'

train_df, valid_df, test_df = read_dataset()
TEST_EMBEDDING_PATH = CFG.EMB_PATH_PREFIX + CFG.MODEL_PATH[:-3] + '_test_embed.csv'
test_embeddings = np.loadtxt(TEST_EMBEDDING_PATH, delimiter=',')
VALID_EMBEDDING_PATH = CFG.EMB_PATH_PREFIX + CFG.MODEL_PATH[:-3] + '_valid_embed.csv'
valid_embeddings = np.loadtxt(VALID_EMBEDDING_PATH, delimiter=',')

search_inb_threshold(valid_df,valid_embeddings)
print(f'CFG.BEST_THRESHOLD is {CFG.BEST_THRESHOLD}')
print(f'CFG.BEST_THRESHOLD_MIN2 is {CFG.BEST_THRESHOLD_MIN2}')
new_emb_1 = blend_neighborhood(test_df,test_embeddings)

Searching best threshold...
threshold = 0.1 -> f1 score = 0.651386550987454, recall = 0.537536326403047, precision = 0.9984934761544488
threshold = 0.11 -> f1 score = 0.6572758079011315, recall = 0.544714680709318, precision = 0.9977293743390986
threshold = 0.12 -> f1 score = 0.6613346576509026, recall = 0.5500509190138452, precision = 0.9972319085985317
threshold = 0.13 -> f1 score = 0.6666907728046472, recall = 0.5564532263479456, precision = 0.9970806873566402
threshold = 0.14 -> f1 score = 0.6716011619082689, recall = 0.5626718029086932, precision = 0.9964970081264433
threshold = 0.15 -> f1 score = 0.6768141054407942, recall = 0.5694686890118738, precision = 0.9958448447935966
threshold = 0.16 -> f1 score = 0.6816470770416911, recall = 0.575620988168598, precision = 0.9956678985588316
threshold = 0.17 -> f1 score = 0.6875378158275254, recall = 0.5834002055983122, precision = 0.9952940118073715
threshold = 0.18 -> f1 score = 0.6927104547909798, recall = 0.5901599493985262, precision

In [22]:
new_valid_emb = blend_neighborhood(valid_df,valid_embeddings)

search_inb_threshold(valid_df,new_valid_emb)
print(f'CFG.BEST_THRESHOLD is {CFG.BEST_THRESHOLD}')
print(f'CFG.BEST_THRESHOLD_MIN2 is {CFG.BEST_THRESHOLD_MIN2}')
new_emb_2 = blend_neighborhood(test_df,new_emb_1)

Test f1 score = 0.7666397422845048, recall = 0.7509019024382299, precision = 0.877955920038871
Searching best threshold...
threshold = 0.1 -> f1 score = 0.7723431979269542, recall = 0.7415952940806982, precision = 0.9181988347413951
threshold = 0.11 -> f1 score = 0.7725851124428161, recall = 0.7438261877120039, precision = 0.9157314577070623
threshold = 0.12 -> f1 score = 0.7723824916906825, recall = 0.7458657887105155, precision = 0.9130431950739175
threshold = 0.13 -> f1 score = 0.7722255399017823, recall = 0.7473533327544436, precision = 0.9108535246033288
threshold = 0.14 -> f1 score = 0.7719297109752793, recall = 0.7487456382237689, precision = 0.9087045652392945
threshold = 0.15 -> f1 score = 0.7714129093678588, recall = 0.7494918347570287, precision = 0.906850275830486
threshold = 0.16 -> f1 score = 0.7713466672032094, recall = 0.7506384290849104, precision = 0.9051414824688422
threshold = 0.17 -> f1 score = 0.7707225846086494, recall = 0.7524926995457174, precision = 0.90173989

In [23]:
new_valid_emb = blend_neighborhood(valid_df,new_valid_emb)

search_inb_threshold(valid_df,new_valid_emb)
print(f'CFG.BEST_THRESHOLD is {CFG.BEST_THRESHOLD}')
print(f'CFG.BEST_THRESHOLD_MIN2 is {CFG.BEST_THRESHOLD_MIN2}')
new_emb_3 = blend_neighborhood(test_df,new_emb_2)

Test f1 score = 0.7732590703421133, recall = 0.7465993165359147, precision = 0.9088691420405415
Searching best threshold...
threshold = 0.1 -> f1 score = 0.7723135898388774, recall = 0.7655561045567136, precision = 0.8881274439344194
threshold = 0.11 -> f1 score = 0.7725828714620111, recall = 0.7663484670558092, precision = 0.88787665160259
threshold = 0.12 -> f1 score = 0.7723028858395677, recall = 0.7669027506271838, precision = 0.886699999070499
threshold = 0.13 -> f1 score = 0.7724158728606898, recall = 0.7680628370680004, precision = 0.8856658865486499
threshold = 0.14 -> f1 score = 0.7725652356240954, recall = 0.7689213610802285, precision = 0.8851552769158829
threshold = 0.15 -> f1 score = 0.7724361088204086, recall = 0.7694569227229616, precision = 0.8846358281032632
threshold = 0.16 -> f1 score = 0.7716686560213921, recall = 0.7697836810693537, precision = 0.8832562699356811
threshold = 0.17 -> f1 score = 0.7715089022212805, recall = 0.7703410722220589, precision = 0.882655829